Loading the fine-tuned model

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import cv2
import os
import imutils

In [69]:
def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    if len(cnts) == 0:
        return img  
    c = max(cnts, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 5
    y1, y2 = max(0, extTop[1] - ADD_PIXELS), min(img.shape[0], extBot[1] + ADD_PIXELS)
    x1, x2 = max(0, extLeft[0] - ADD_PIXELS), min(img.shape[1], extRight[0] + ADD_PIXELS)
    new_img = img[y1:y2, x1:x2].copy()
    return new_img


In [ ]:
def preprocess(img,img_size=224):
            img = crop_img(img)
            img = cv2.resize(img, (img_size, img_size))
            img = cv2.bilateralFilter(img, 2, 50, 50)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = np.uint8(img)
            img = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
            return img/255.0

In [ ]:
class GradCAM:
    """
    Produces class activation map using the gradient information
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        self.register_hooks()
        
    def register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
            
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()
        
        # Register the hooks
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_full_backward_hook(backward_hook)
    
    def generate_cam(self, input_image, target_class=None):
        # Forward pass
        model_output = self.model(input_image)
        
        if target_class is None:
            target_class = torch.argmax(model_output, dim=1).item()
        
        # Zero gradients
        self.model.zero_grad()
        
        # Target for backprop
        one_hot = torch.zeros_like(model_output)
        one_hot[0, target_class] = 1
        
        # Backward pass
        model_output.backward(gradient=one_hot, retain_graph=True)
        
        # Get weights
        gradients = self.gradients.cpu().data.numpy()[0]
        
        # Take the average of gradients over each channel (global average pooling)
        weights = np.mean(gradients, axis=(1, 2))
        
        # Get activation maps
        activations = self.activations.cpu().data.numpy()[0]
        
        # Create weighted sum of activation maps
        cam = np.zeros(activations.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * activations[i]
        
        # Apply ReLU
        cam = np.maximum(cam, 0)
        
        # Normalize the CAM
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-8)
        
        return cam, target_class

# Load and prepare image
image_path = "Preprocessed_Dataset\Testing\glioma\Te-gl_0013.jpg"
img = cv2.imread(image_path)
img = preprocess(img)

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# Preprocess the image
input_tensor = transform(img).unsqueeze(0)

# Define model class
class BrainTumorClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BrainTumorClassifier, self).__init__()
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        for param in self.base_model.parameters():
            param.requires_grad = True
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_features, num_classes)
            # Note: removed Softmax here, as it can interfere with gradient flow
        )

    def forward(self, x):
        return self.base_model(x)

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BrainTumorClassifier(num_classes=4)

checkpoint = torch.load('./Model/resnet-50_finetuned.pth', map_location=device)

# If your checkpoint contains 'model_state_dict' key
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)

model.eval()
model = model.to(device)
input_tensor = input_tensor.to(device)

# Get the target layer - last convolutional layer
target_layer = model.base_model.layer4[-1].conv3

# Initialize GradCAM
grad_cam = GradCAM(model, target_layer)

# Generate CAM
cam, pred_idx = grad_cam.generate_cam(input_tensor)

# Resize CAM to match image size
cam_resized = cv2.resize(cam, (224, 224))

# Class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
predicted_class = class_names[pred_idx]

# Create heatmap
heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

# Convert original image to RGB if it's not
if len(original_img.shape) == 2:  # Grayscale
    original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_GRAY2RGB)
else:
    original_img_rgb = original_img

# Superimpose heatmap on original image
alpha = 0.6  # Transparency factor
superimposed_img = heatmap * alpha + original_img_rgb * (1 - alpha)
superimposed_img = superimposed_img / np.max(superimposed_img)  # Normalize

# Visualize
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(original_img_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cam_resized, cmap='jet')
plt.title('Grad-CAM Heatmap')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.title(f'Superimposed: {predicted_class}')
plt.axis('off')

plt.tight_layout()
plt.savefig('improved_gradcam_result.png', bbox_inches='tight', dpi=300)
plt.show()

# Try another visualization approach with cv2
heatmap_img = cv2.resize(heatmap, (original_img_rgb.shape[1], original_img_rgb.shape[0]))
cv2_superimposed = cv2.addWeighted(original_img_rgb, 0.6, heatmap_img, 0.4, 0)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_img_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2_superimposed)
plt.title(f'CV2 Superimposed: {predicted_class}')
plt.axis('off')

plt.tight_layout()
plt.savefig('cv2_gradcam_result.png', bbox_inches='tight', dpi=300)
plt.show()

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'